In [8]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model(r'D:\MTECH\PROJECT\CV\FINAL - ORIGINAL\BEST_MODEL.h5')

# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(r'D:\MTECH\PROJECT\DM\SECRET\CASCADE\haarcascade_frontalface_default.xml')

# Define emotions
emotions = ['angerH', 'angerL', 'happyH', 'happyL', 'sadH', 'sadL', 'surprise']

# Function to preprocess face image
def preprocess_face(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.5, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face = gray_image[y:y + h, x:x + w]
        resized_face = cv2.resize(face, (32, 32), interpolation=cv2.INTER_LINEAR)
        normalized_face = resized_face / 255.0
        lbp = local_binary_pattern(resized_face, 8, 1, method='uniform')
        resized_lbp = cv2.resize(lbp, (32, 32), interpolation=cv2.INTER_LINEAR)
        flattened_face = normalized_face.flatten()
        flattened_lbp = resized_lbp.flatten()
        combined_feature = np.concatenate([flattened_face, flattened_lbp])
        return combined_feature.reshape(1, 32, 32, 2)
    else:
        return None

# Open a video capture object
cap = cv2.VideoCapture(1)
# Loop over frames from the video stream
while True:
    # Read the frame from the video capture object
    ret, frame = cap.read()
    
    # Preprocess the frame and make a prediction
    face_input = preprocess_face(frame)
    
    if face_input is not None:
        # Make prediction using the model
        prediction = model.predict(face_input)
        predicted_label = np.argmax(prediction)
        emotion = emotions[predicted_label]
        
        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5, minSize=(30, 30))
        
        # Draw bounding boxes around detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Display the frame
    cv2.imshow('Real-time Facial Emotion Recognition', frame)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 18ms/step
